In [ ]:
%load_ext autoreload
%autoreload 2

In [140]:
import pickle
from ares.databases.annotation_database import AnnotationDatabase, TEST_ANNOTATION_DB_PATH
from ares.image_utils import load_video_frames

from ares.app.annotation_helpers import draw_annotations
import matplotlib.pyplot as plt
from ares.configs.annotations import Annotation
import numpy as np


target_fps = 1
db = AnnotationDatabase(connection_string=TEST_ANNOTATION_DB_PATH)
peek = db.peek_database()

In [ ]:
peek['videos'][3]

In [142]:
from collections import defaultdict

def display_annotations(annotations, frames, frame_indices):
    annotation_dict = defaultdict(list)
    for annotation in annotations:
        frame_num = int(annotation["key"].split("_")[1]) # "frame_N_annotation"
        annotation_dict[frame_num].append(annotation)

    shared_frame_indices = sorted(set(annotation_dict.keys()) & set(frame_indices))
    side_len = int(np.ceil(np.sqrt(len(shared_frame_indices))))  # Calculate grid dimensions

    # Create a figure with subplots in a square grid
    fig = plt.figure(figsize=(15, 15))
    for i, frame_idx in enumerate(shared_frame_indices):
        frame_anns = [Annotation(**ann['value']) for ann in annotation_dict[frame_idx]]
        frame_num = frame_indices.index(frame_idx)
        im = draw_annotations(frames[frame_num], frame_anns)
        
        # Create subplot in grid
        plt.subplot(side_len, side_len, i + 1)
        plt.title(f"Frame idx {frame_idx}")
        plt.axis('off')
        plt.imshow(im)

    plt.tight_layout()
    plt.show()

In [143]:
def get_and_display(dataset_name, fname, target_fps):
    frames, frame_indices = load_video_frames(dataset_name, fname, target_fps)
    video_id = f"{dataset_name}/{fname}"
    annotations = db.get_annotations(video_id=video_id)
    video_data = db.get_video_metadata(video_id=video_id)
    print(video_data['label_str'])
    display_annotations(annotations, frames, frame_indices)

In [144]:
# dataset_name = "cmu_play_fusion"
# fname = "data/train/episode_208.mp4"

dataset_name = "ucsd_kitchen_dataset_converted_externally_to_rlds"
fnames = [f"data/train/episode_{i}.mp4" for i in range(10, 20)]


In [ ]:
for fname in fnames: 
    get_and_display(dataset_name, fname, target_fps)